In [36]:
import struct
from io import BytesIO

# --- Binary helpers ---
def read_byte(f): return struct.unpack("<B", f.read(1))[0]
def read_bool(f): return struct.unpack("<?", f.read(1))[0]
def read_int(f): return struct.unpack("<i", f.read(4))[0]
def read_short(f): return struct.unpack("<h", f.read(2))[0]
def read_long(f): return struct.unpack("<q", f.read(8))[0]
def read_float(f): return struct.unpack("<f", f.read(4))[0]
def read_double(f): return struct.unpack("<d", f.read(8))[0]

def read_uleb128(f):
    result = 0
    shift = 0
    while True:
        b = read_byte(f)
        result |= (b & 0x7f) << shift
        if (b & 0x80) == 0:
            break
        shift += 7
    return result

def read_string(f):
    marker = read_byte(f)
    if marker != 0x0b:  # 0x00 = empty string
        return ""
    length = read_uleb128(f)
    return f.read(length).decode("utf-8", errors="ignore")

# --- Star rating helper ---
def read_star_pairs(f, double=False):
    n = read_int(f)
    pairs = []
    for _ in range(n):
        marker = read_byte(f)
        if marker != 0x08:
            raise ValueError(f"Unexpected marker for star pair: {marker}")
        mod = read_int(f)
        marker2 = read_byte(f)
        if double:
            if marker2 != 0x0d:
                raise ValueError(f"Expected 0x0d for double star value, got {marker2}")
            value = read_double(f)
        else:
            if marker2 != 0x0c:
                raise ValueError(f"Expected 0x0c for float star value, got {marker2}")
            value = read_float(f)
        pairs.append((mod, value))
    return pairs

# --- Timing points helper ---
def read_timing_points(f):
    n = read_int(f)
    points = []
    for _ in range(n):
        bpm = read_double(f)
        offset = read_double(f)
        uninherited = read_bool(f)
        points.append({
            "bpm": bpm,
            "offset": offset,
            "uninherited": uninherited
        })
    return points

def parse_beatmap(buf, version):
    beatmap = {}
    beatmap["artist"] = read_string(buf)
    beatmap["artist_unicode"] = read_string(buf)
    beatmap["title"] = read_string(buf)
    beatmap["title_unicode"] = read_string(buf)
    beatmap["creator"] = read_string(buf)
    beatmap["diff_name"] = read_string(buf)
    beatmap["audio_file"] = read_string(buf)
    beatmap["md5_hash"] = read_string(buf)
    beatmap["filename"] = read_string(buf)

    beatmap["ranked_status"] = read_byte(buf)
    beatmap["nr_hitcircles"] = read_short(buf)
    beatmap["nr_sliders"] = read_short(buf)
    beatmap["nr_spinners"] = read_short(buf)

    beatmap["last_modification_time"] = read_long(buf)

    #Doesnt work before version 20140609
    beatmap["ar"] = read_float(buf)
    beatmap["cs"] = read_float(buf)
    beatmap["hp"] = read_float(buf)
    beatmap["od"] = read_float(buf)

    beatmap["slider_velocity"] = read_double(buf)

    if version>=20250815:
        star_rating_double = False
    else:
        star_rating_double = True

    beatmap["std_stars"] = read_star_pairs(buf, double=star_rating_double)
    beatmap["taiko_stars"] = read_star_pairs(buf, double=star_rating_double)
    beatmap["ctb_stars"] = read_star_pairs(buf, double=star_rating_double)
    beatmap["mania_stars"] = read_star_pairs(buf, double=star_rating_double)

    beatmap["drain_time"] = read_int(buf)
    beatmap["total_time"] = read_int(buf)
    beatmap["preview_time"] = read_int(buf)

    beatmap["timing_points"] = read_timing_points(buf)

    beatmap["diff_id"] = read_int(buf)
    beatmap["beatmap_id"] = read_int(buf)
    beatmap["thread_id"] = read_int(buf)

    beatmap["grade_std"] = read_byte(buf)
    beatmap["grade_taiko"] = read_byte(buf)
    beatmap["grade_ctb"] = read_byte(buf)
    beatmap["grade_mania"] = read_byte(buf)

    beatmap["local_offset"] = read_short(buf)
    beatmap["stack_leniency"] = read_float(buf)
    beatmap["mode"] = read_byte(buf)

    beatmap["source"] = read_string(buf)
    beatmap["tags"] = read_string(buf)
    beatmap["online_offset"] = read_short(buf)
    beatmap["font"] = read_string(buf)

    beatmap["unplayed"] = read_bool(buf)
    beatmap["last_played"] = read_long(buf)
    beatmap["osz2"] = read_bool(buf)
    beatmap["folder_name"] = read_string(buf)
    beatmap["last_checked"] = read_long(buf)
    beatmap["ignore_sound"] = read_bool(buf)
    beatmap["ignore_skin"] = read_bool(buf)
    beatmap["disable_storyboard"] = read_bool(buf)
    beatmap["disable_video"] = read_bool(buf)
    beatmap["visual_override"] = read_bool(buf)

    beatmap["last_mod_time"] = read_int(buf)
    beatmap["mania_scroll_speed"] = read_byte(buf)

    return beatmap


In [37]:
def parse_osudb(file_path):
    with open(file_path, "rb") as f:
        buf = BytesIO(f.read())

    osu_version = read_int(buf)
    folder_count = read_int(buf)
    account_unlocked = read_bool(buf)
    unlock_date = read_long(buf)
    player_name = read_string(buf)

    num_beatmaps = read_int(buf)
    print(f"Total of {num_beatmaps} beatmaps")
    beatmaps = [parse_beatmap(buf, osu_version) for _ in range(num_beatmaps)]
    return {
        "version": osu_version,
        "player": player_name,
        "beatmaps": beatmaps
    }

In [70]:
import win32gui
import re
import time
for i in range(10):
    thetitle = win32gui.GetForegroundWindow()
    print(thetitle)
    time.sleep(0.2)

6557732
6557732
6557732
6557732
6557732
6557732
6557732
6557732
6557732
6557732


In [75]:
db = parse_osudb("E:\\osu!\\osu!.db")
print(f"Found {len(db['beatmaps'])} beatmaps")
#first = db['beatmaps'][0]
#print(first["artist"], "-", first["title"], "[", first["diff_name"], "]")
#print("CS:", first["cs"], "AR:", first["ar"], "HP:", first["hp"], "OD:", first["od"])
#print("First timing point:", first["timing_points"][0] if first["timing_points"] else "None")

Total of 126734 beatmaps
Found 126734 beatmaps


In [98]:
pattern = r"^(?P<artist>.+?)\s*-\s*(?P<songtitle>.+?)\s*\[(?P<diffname>.+)\]$"
for i in range(20):
    window_title = win32gui.GetWindowText(win32gui.GetForegroundWindow())
    if window_title[:4] == "osu!":
        print(window_title[8:])
        song = window_title[8:]
        match = re.match(pattern, song)

        if match:
            artist = match.group("artist")
            songtitle = match.group("songtitle")
            diffname = match.group("diffname")
        md5_hashes = [b['md5_hash'] for b in db['beatmaps'] if b['title']==songtitle and b['artist'] == artist and b['diff_name'] == diffname]
    time.sleep(0.5)

Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]
Yousei Teikoku - Senketsu no Chikai [Insanity]


In [99]:
songtitle, artist

('Senketsu no Chikai', 'Yousei Teikoku')

In [100]:
[b['md5_hash'] for b in db['beatmaps'] if b['title']==songtitle and b['artist'] == artist and b['diff_name'] == diffname]

['92437112319516d39b458898ae95a840']

In [102]:
md5_pattern = re.compile(rb"\b[a-f0-9]{32}\b")
md5_hash = '92437112319516d39b458898ae95a840'
